In [5]:
import os
os.chdir("/Users/georg/Desktop/chess-recognition")

In [6]:
import torch
import torchvision
from torchvision import transforms as T
from torch import nn, optim
import torch.nn.functional as F
from pathlib import Path

from chesscog import DATA_DIR

In [13]:
TRANSFORM_IMG = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

dataset = torchvision.datasets.ImageFolder(root=DATA_DIR / "occupancy",
                                           transform=TRANSFORM_IMG)
num_train = int(0.9 * len(dataset))
train_set, val_set = torch.utils.data.random_split(dataset,
                                                   (num_train,
                                                    len(dataset) - num_train),
                                                   generator=torch.Generator().manual_seed(42))

train_loader = torch.utils.data.DataLoader(train_set, batch_size=4,
                                           shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=4,
                                         shuffle=False, num_workers=2)

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input size: 100x100
        self.conv1 = nn.Conv2d(3, 16, 5) # 96
        self.pool1 = nn.MaxPool2d(2, 2) # 48
        self.conv2 = nn.Conv2d(16, 32, 5) # 44
        self.pool2 = nn.MaxPool2d(2, 2) # 22
        self.conv3 = nn.Conv2d(32, 64, 3) # 20
        self.pool3 = nn.MaxPool2d(2, 2) # 10
        self.fc1 = nn.Linear(64 * 10 * 10, 1000)
        self.fc2 = nn.Linear(1000, 256)
        self.fc3 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 10 * 10)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [19]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

00, 100])
torch.Size([4, 16, 48, 48])
torch.Size([4, 32, 22, 22])
torch.Size([4, 64, 10, 10])
torch.Size([4, 3, 100, 100])
torch.Size([4, 16, 48, 48])
torch.Size([4, 32, 22, 22])
torch.Size([4, 64, 10, 10])
torch.Size([4, 3, 100, 100])
torch.Size([4, 16, 48, 48])
torch.Size([4, 32, 22, 22])
torch.Size([4, 64, 10, 10])
torch.Size([4, 3, 100, 100])
torch.Size([4, 16, 48, 48])
torch.Size([4, 32, 22, 22])
torch.Size([4, 64, 10, 10])
torch.Size([4, 3, 100, 100])
torch.Size([4, 16, 48, 48])
torch.Size([4, 32, 22, 22])
torch.Size([4, 64, 10, 10])
torch.Size([4, 3, 100, 100])
torch.Size([4, 16, 48, 48])
torch.Size([4, 32, 22, 22])
torch.Size([4, 64, 10, 10])
torch.Size([4, 3, 100, 100])
torch.Size([4, 16, 48, 48])
torch.Size([4, 32, 22, 22])
torch.Size([4, 64, 10, 10])
torch.Size([4, 3, 100, 100])
torch.Size([4, 16, 48, 48])
torch.Size([4, 32, 22, 22])
torch.Size([4, 64, 10, 10])
torch.Size([4, 3, 100, 100])
torch.Size([4, 16, 48, 48])
torch.Size([4, 32, 22, 22])
torch.Size([4, 64, 10, 10])
to

KeyboardInterrupt: 